In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(device)


cpu


In [ ]:
lass Discriminator(nn.Module):
    """Discriminator network with PatchGAN."""
    def __init__(self, 
                input_channels: int = 3,
                patch_size: int = 70,
                patch_stride: int = 40,):
        super().__init__()

        self.create_patches = nn.Conv2d(input_channels, 64,
                        kernel_size =  patch_size,
                        stride = patch_stride)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.create_patches(x)

        return x

input = torch.randn(1, 3, 256, 256)
D = Discriminator().to(device)
D(input).shape